In [2]:
!pip install pygame

     |████████████████████████████████| 14.0 MB 5.5 MB/s eta 0:00:01


In [4]:
## Google's Generative AI search provided the initial draft of this code. 
## I have modified it since. 
## It provided these sources: 
## https://github.com/jk96491/RL_Algorithms?tab=readme-ov-file 
## https://github.com/abyaadrafid/Deep-Reinforcement-Learning 
## I couldn't find a draft of this code in the sources, 
## so Generative AI search may have generated the code. 

import random 
import gym
import torch
import torch.nn as nn
import torch.optim as optim 

class Object(object):
    pass

class ReplayBuffer(): 
    def __init__(self, capacity=10000): 
        self.n = 0 
        self.capacity = capacity 
        self.state_list = [] 
        self.action_list = [] 
        self.reward_list = [] 
        self.next_state_list = [] 
        self.done_list = [] 
        pass 
    def add(self, state, action, reward, next_state, done): 
        if self.n >= self.capacity: 
            ## discard earliest observation 
            self.state_list = self.state_list[1:] 
            self.action_list = self.action_list[1:] 
            self.reward_list = self.reward_list[1:] 
            self.next_state_list = self.next_state_list[1:] 
            self.done_list = self.done_list[1:] 
            self.n -= 1 
        pass 
        ## cast to torch  
        state = torch.tensor(state) 
        action = torch.tensor(action) 
        reward = torch.tensor(reward) 
        next_state = torch.tensor(next_state) 
        done = torch.tensor(done) 
        ## append to buffer 
        self.state_list.append(state) 
        self.action_list.append(action) 
        self.reward_list.append(reward) 
        self.next_state_list.append(next_state) 
        self.done_list.append(done) 
        self.n += 1 
        pass 
    def sample(self, batch_size=32): 
        ## sample lists 
        out = Object() ## transitions 
        out.sate = [] 
        out.action = [] 
        out.reward = [] 
        out.next_state = [] 
        out.done = [] 
        for _ in range(batch_size): 
            idx = random.randint(0, self.n) 
            out.state.append(self.state_list[idx]) 
            out.action.append(self.action_list[idx]) 
            out.reward.append(self.reward_list[idx]) 
            out.next_state.append(self.next_state_list[idx]) 
            out.done.append(self.done_list[idx]) 
            pass 
        ## stack  
        out.state = torch.stack(out.state) 
        out.action = torch.stack(out.action) 
        out.reward = torch.stack(out.reward) 
        out.next_state = torch.stack(out.next_state) 
        out.done = torch.stack(out.done) 
        return out 
    pass 

# Define the actor and critic networks
class Actor(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(Actor, self).__init__()

        self.fc1 = nn.Linear(state_dim, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, action_dim)

    def forward(self, state):
        x = torch.relu(self.fc1(state))
        x = torch.relu(self.fc2(x))
        action = torch.tanh(self.fc3(x))

        return action

class Critic(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(Critic, self).__init__()

        self.fc1 = nn.Linear(state_dim + action_dim, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 1)

    def forward(self, state, action):
        x = torch.cat([state, action], dim=1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        value = self.fc3(x)

        return value

# Create the actor and critic networks
actor = Actor(state_dim=4, action_dim=1)
critic = Critic(state_dim=4, action_dim=1)

# Define the optimizers
actor_optimizer = optim.Adam(actor.parameters(), lr=1e-3)
critic_optimizer = optim.Adam(critic.parameters(), lr=1e-3)

# Create the replay buffer
replay_buffer = ReplayBuffer(capacity=100000) 

# Define the environment
env = gym.make('CartPole-v1')

# Train the agent
for episode in range(1000):
    state = env.reset()

    for t in range(1000):
        action = actor(torch.tensor(state)) 
        
        action_p = action.item() * .5 + .5 
        action_int = ## must be 0 or 1 
        next_state, reward, done, _ = env.step(action)

        replay_buffer.add(state, action, reward, next_state, done)

        if len(replay_buffer) > 1000:
            # Sample a batch of transitions from the replay buffer
            transitions = replay_buffer.sample(batch_size=32)

            # Calculate the target Q-values
            target_Q = critic(transitions.next_state, actor(transitions.next_state))
            target_Q = target_Q.detach()

            # Calculate the current Q-values
            current_Q = critic(transitions.state, transitions.action)

            # Calculate the critic loss
            critic_loss = torch.mean((target_Q - current_Q).pow(2))

            # Calculate the actor loss
            actor_loss = -torch.mean(critic(transitions.state, actor(transitions.state)))

            # Update the critic network
            critic_optimizer.zero_grad()
            critic_loss.backward()
            critic_optimizer.step()

            # Update the actor network
            actor_optimizer.zero_grad()
            actor_loss.backward()
            actor_optimizer.step()

        state = next_state

        if done:
            break

    # Evaluate the agent
    episode_reward = 0

    for t in range(1000):
        state = env.reset()

        action = actor(state)

        next_state, reward, done, _ = env.step(action)

        episode_reward += reward

        if done:
            break

    print('Episode {}: {}'.format(episode, episode_reward))

AssertionError: tensor([-0.0676], grad_fn=<TanhBackward0>) (<class 'torch.Tensor'>) invalid

In [10]:
import numpy as np 
np.random.binomial(1, .5) 

tensor([-0.0676], grad_fn=<TanhBackward0>)